In [1]:
from nltk.tokenize import word_tokenize
import sqlite3


conn = sqlite3.connect('news.db')
cur = conn.cursor()

query = """
    SELECT id, url, content, date
    FROM news;
"""
cur.execute(query)
corpus = cur.fetchall()
corpus = [news[2] for news in corpus]

In [2]:
from konlpy.tag import Kkma
from collections import defaultdict


pos_tag = Kkma().pos

kkmaTDM = defaultdict(lambda: defaultdict(int))
for i, d in enumerate(corpus):
    for t in [_[0] for _ in pos_tag(d)
             if _[1].startswith("N") or _[1].startswith("V")
             if len(_[0]) > 1]:
        kkmaTDM[t][i] += 1

C:\Users\u\Anaconda3\lib\site-packages\jpype\_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


In [3]:
from konlpy.tag import Hannanum
from collections import defaultdict


pos_tag = Hannanum().pos

hanTDM = defaultdict(lambda: defaultdict(int))
for i, d in enumerate(corpus):
    for t in [_[0] for _ in pos_tag(d)
             if _[1] in ['N', 'P']
             if len(_[0]) > 1]:
        hanTDM[t][i] += 1

In [4]:
# sorted([{t: len(docList)} for t, docList in kkmaTDM.items()],
#        key=lambda x: sum(x.values()), reverse=True)

In [5]:
from math import log2


pSeed = ['노력', '한국', '평화']
nSeed = ['규제', '일본', '비판']

N = len(corpus)

pSeedDF = list(kkmaTDM[pSeed[0]])
nSeedDF = list(kkmaTDM[nSeed[0]])

pProbY = len(pSeedDF) / N
nProbY = len(nSeedDF) / N

posPMI = defaultdict(float)
negPMI = defaultdict(float)

posNormPMI = defaultdict(float)
negNormPMI = defaultdict(float)

for t in kkmaTDM:
    termDF = list(kkmaTDM[t])
    pX = len(termDF) / N
    posXY = (len(set(pSeedDF).intersection(termDF)) + 1) / N
    negXY = (len(set(nSeedDF).intersection(termDF)) + 1) / N
    
    posPMI[t] = log2(posXY / (pX + pProbY))
    negPMI[t] = log2(negXY / (pX + nProbY))
    
    posNormPMI[t] = posXY / -log2(posXY)
    negNormPMI[t] = negXY / -log2(negXY)
    
#     # p(x ㅌ X) = df(t), t ㅌ V(kkmaTDM.keys) / N
#     # p(y ㅌ Y) = df(pSeed[0]), y ㅌ V(kkmaTDM.keys) / N
#     # P(x, y) = df(t) - df(pSeed[0]) / N

In [6]:
sorted(posPMI.items(), key=lambda x: x[1], reverse=True)[:10]

[('노력', -0.5849625007211563),
 ('남방', -0.5849625007211563),
 ('스틸웰', -0.7369655941662062),
 ('동맹', -0.7369655941662062),
 ('한미', -0.7369655941662062),
 ('현종', -0.7369655941662062),
 ('데이', -0.7369655941662062),
 ('비드', -0.7369655941662062),
 ('신임', -0.7369655941662062),
 ('동아시아', -0.7369655941662062)]

In [7]:
SO = defaultdict(float)
nSO = defaultdict(float)

for t in kkmaTDM:
    SO[t] = posPMI[t] - negPMI[t]
    nSO[t] = posNormPMI[t] - negNormPMI[t]

In [8]:
sorted(nSO.items(), key=lambda x:x[1], reverse=True)[:10],
sorted(nSO.items(), key=lambda x:x[1])[:10]

[('규제', -0.1969870030478118),
 ('지나', -0.14493994627171375),
 ('뉴스', -0.14358426495672572),
 ('기자', -0.14358426495672572),
 ('정부', -0.12706765645682155),
 ('아니', -0.12706765645682155),
 ('경제', -0.12706765645682155),
 ('한국', -0.11526124895155697),
 ('일본', -0.11526124895155697),
 ('위하', -0.11526124895155697)]

In [9]:
pSeed = ['노력', '한국', '평화']
nSeed = ['규제', '일본', '비판']

N = len(corpus)

for i in range(len(pSeed)):
    pSeedDF = list(kkmaTDM[pSeed[i]])
    nSeedDF = list(kkmaTDM[nSeed[i]])

    pProbY = len(pSeedDF) / N
    nProbY = len(nSeedDF) / N

    posPMI = defaultdict(lambda:defaultdict(float))
    negPMI = defaultdict(lambda:defaultdict(float))

    posNormPMI = defaultdict(lambda:defaultdict(float))
    negNormPMI = defaultdict(lambda:defaultdict(float))

    for t in kkmaTDM:
        termDF = list(kkmaTDM[t])
        pX = len(termDF) / N
        posXY = (len(set(pSeedDF).intersection(termDF)) + 1) / N
        negXY = (len(set(nSeedDF).intersection(termDF)) + 1) / N

        posPMI[pSeed[i]][t] = log2(posXY / (pX + pProbY))
        negPMI[pSeed[i]][t] = log2(negXY / (pX + nProbY))

        posNormPMI[pSeed[i]][t] = posPMI[pSeed[i]][t] / -log2(posXY)
        negNormPMI[nSeed[i]][t] = negPMI[nSeed[i]][t] / -log2(negXY)

In [10]:
SO = defaultdict(float)
nSO = defaultdict(float)

for t in kkmaTDM:
    for i in range(len(pSeed)):
        SO[t] += posPMI[pSeed[i]][t] - negPMI[nSeed[i]][t]
        nSO[t] += posNormPMI[pSeed[i]][t] - negNormPMI[nSeed[i]][t]

In [11]:
sorted(nSO.items(), key=lambda x:x[1], reverse=True)[:10],
sorted(nSO.items(), key=lambda x:x[1])[:10]

[('기자', -1.0),
 ('17', -0.9463946303571861),
 ('뉴스', -0.9463946303571861),
 ('대하', -0.9266284080291269),
 ('무단', -0.8842282173954807),
 ('배포', -0.8842282173954807),
 ('금지', -0.8842282173954807),
 ('네이버', -0.8738421406894301),
 ('위하', -0.8532568160582538),
 ('구독', -0.8532568160582538)]

In [12]:
pos_tag = Kkma().pos

print('1번그룹:', ', '.join(pSeed))
print('2번그룹:', ', '.join(nSeed))
print()

for i, d in enumerate(corpus):
    docSO = 0.
    docNSO = 0.

    for t in [_[0] for _ in pos_tag(d)
              if _[1].startswith("N") or _[1].startswith("V")
              if len(_[0]) > 1]:
        docSO += SO[t]
        docNSO += nSO[t]
    print(f'{i+1}번째 문서: SO={docSO}|{1 if docSO > 0 else 0}, NSO={docNSO}|{1 if docNSO > 0 else 2}')

1번그룹: 노력, 한국, 평화
2번그룹: 규제, 일본, 비판

1번째 문서: SO=-1135.2086377034666|0, NSO=-245.08583555431738|2
2번째 문서: SO=-1135.2086377034666|0, NSO=-245.08583555431738|2
3번째 문서: SO=-624.1898360187596|0, NSO=-160.98752119325593|2
4번째 문서: SO=-560.3825024680659|0, NSO=-120.59128988761321|2
5번째 문서: SO=-1200.730230664284|0, NSO=-257.4234661076898|2
6번째 문서: SO=-1055.2686302516704|0, NSO=-224.37725115005438|2
7번째 문서: SO=-279.29549239561817|0, NSO=-58.03169711844651|2
8번째 문서: SO=-607.8409135659078|0, NSO=-144.28044186931717|2
9번째 문서: SO=-815.1404187447162|0, NSO=-180.51087918161645|2
10번째 문서: SO=-712.510958101813|0, NSO=-151.26590901095693|2
11번째 문서: SO=-811.1448163005731|0, NSO=-202.3471724490118|2
12번째 문서: SO=-1269.0634680356181|0, NSO=-264.2064749668834|2
13번째 문서: SO=-976.337887165768|0, NSO=-202.3334506520634|2
14번째 문서: SO=-792.4805264051308|0, NSO=-162.8790963555916|2
15번째 문서: SO=-1051.625110703815|0, NSO=-221.68641952537664|2
16번째 문서: SO=-1216.5614891509088|0, NSO=-261.03271253916444|2
17번째 문서: SO=-548

In [13]:
sorted(nSO.items(), key=lambda x:x[1])[:30]

[('기자', -1.0),
 ('17', -0.9463946303571861),
 ('뉴스', -0.9463946303571861),
 ('대하', -0.9266284080291269),
 ('무단', -0.8842282173954807),
 ('배포', -0.8842282173954807),
 ('금지', -0.8842282173954807),
 ('네이버', -0.8738421406894301),
 ('위하', -0.8532568160582538),
 ('구독', -0.8532568160582538),
 ('전재', -0.8532568160582538),
 ('밝히', -0.8518938829506677),
 ('지나', -0.8368288369533895),
 ('한국', -0.8197015881601898),
 ('말하', -0.8154648767857288),
 ('따르', -0.8018128829347333),
 ('보이', -0.8018128829347333),
 ('나오', -0.7966397701969122),
 ('경제', -0.7906232373022846),
 ('상황', -0.7830918514469462),
 ('설명', -0.7830918514469462),
 ('시간', -0.7788935082334042),
 ('사진', -0.7788935082334042),
 ('아니', -0.7758642925363403),
 ('내용', -0.7556957972346928),
 ('결과', -0.7556957972346928),
 ('일본', -0.7404120793976952),
 ('통하', -0.7404120793976952),
 ('규제', -0.7364429698681167),
 ('20', -0.7364429698681167)]